# Buchbergers Algorithm

## Boiler Plate

First load the Sympy module

In [12]:
## Sympy Boiler Plate

from __future__ import division
from sympy import *
t, x, y, z = symbols('t x y z')  # I believe this order matters, i.e. t<-x1, x <- x2, y <- x3 ...
k, m, n = symbols('k m n', integer=True)
f, g, h = symbols('f g h', cls=Function)
init_printing()
init_printing(use_latex='mathjax', latex_mode='equation')



import pyperclip
def lx(expr):
    pyperclip.copy(latex(expr))
    print(expr)
# NB: See also srepr(), display() and print_mathml()

import numpy as np
import matplotlib as plt

## Terminology

There are mutlidegree's, leading coefficients, leading monomials and leading terms.

In [116]:
def f(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x**2*y**2*z**2
    # return 13*x**2+11*x-7
def g(x, domain = 'QQ'):
    return 2+3*x+5*y+7*x*y+11*x**2*y+13*x*y**2+17*x**2*y**2+19*x*y**2*z**2
    # return 13*x**2+11*x-7


In [117]:
print("Leading Monomial:")
display(LM(f(x)))

print("Leading Coefficient")
display(LC(f(x)))

print("Leading Term")
display(LT(f(x)))

Leading Monomial:


 2  2  2
x ⋅y ⋅z 

Leading Coefficient


19

Leading Term


    2  2  2
19⋅x ⋅y ⋅z 

The initial monomial with respect to an ordering is the first monomial returned by `sympy`, I believe that sympy uses lexicographic ordering under the hood, as suggested by the documentation [^spdocsPolySysRef], but I'd need to get a better understanding of the differences between `lex`, `grlex` and `grevlex` to be certain.


[^spdocsPolySysRef]: SymPy Development Team. “Polynomials Manipulation Module Reference — SymPy 1.8 Documentation.” Sympy Documentation. Accessed April 13, 2021. https://docs.sympy.org/latest/modules/polys/reference.html.

In [118]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

In this case, the first monomial is $x^2y^2z^2$ and so that is the initial. In practice we can just use the `LT`, the initial is just another way to skin the cat, which is probably why sympy doesn't offer a clear way to extract the initial. [^initVlt].


[^initVlt]: If it was necessary to get the initial one option might be to extract the polynomial to a string `str(f(x))` and then use regex `^[A-Za-z0-9*]+(?=[\ +])` to extract the first term and then take the `LM` of that term to get the initial monomial.

In [119]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

In [120]:
g(x)

    2  2       2           2  2         2                        
17⋅x ⋅y  + 11⋅x ⋅y + 19⋅x⋅y ⋅z  + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

## Implementing the Algorithm

### Set of Polynomials

To begin with, we'll implement buchberger's algorithm on a simple polynomial system, the reason for this choice is to see the implementation of the algorithm generally, a *simple* system such as a linear or single variable system, may be more confusing at first, but will be returned to later in order to draw insights.

So we have our bag of polynomials:

In [126]:
F = [x**3-2*x*y, x**2*y-2*y**2 + x]
for poly in F:
    display(poly)

 3        
x  - 2⋅x⋅y

 2            2
x ⋅y + x - 2⋅y 

These polynomials are equal to zero and so could also be expressed:

\begin{align}
x^3 &= 2xy \\
  x &=2y^2-x^2y    
\end{align}

Now we want our algorithm to give us back:

In [124]:
G_out = polys.polytools.groebner(F, x, y, order = 'lex', method = 'buchberger')
G_out


             ⎛⎡       2   3⎤                           ⎞
GroebnerBasis⎝⎣x - 2⋅y , y ⎦, x, y, domain=ℤ, order=lex⎠

Note that the 2nd tupple $\left(x, y\right)$ that is printed out refers to the variables, the first tupple is the reduced Groebner Basis, a clearer way to display it would be as a list:

In [125]:

G = []
for poly in G_out:
    G.append(poly)

display(G)


⎡       2   3⎤
⎣x - 2⋅y , y ⎦

and so overall we have:

$$
\begin{array}{rcl}
x^{3} & = & 2xy\\
x & = & 2y^{2}-x^{2}y
\end{array}\implies\begin{array}{rcl}
y^{2} & = & \frac{1}{2}x\\
y^{3} & = & 0
\end{array}
$$

\begin{align}
y^2 &= \frac{1}{2}x \\
y^3 &= 0
\end{align}    

## S-Polynomial

The $S$-Polynomial used to determine whether or not two polynomials belong in the Groebner Basis and it is given by:

$$
S=\mathrm{LCM}\left(\mathrm{LM}\left(f\right),\mathrm{LM}\left(g\right)\right)\times\left(\frac{f}{\mathrm{LT}\left(f\right)}-\frac{g}{\mathrm{LT}\left(g\right)}\right)
$$

Sympy comes with a function to determine the LCM of polynomials:



In [142]:
from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div
# https://docs.sympy.org/latest/modules/polys/basics.html
LMf = LM(F[0])
LMg = LM(F[1])
# LCM12 = monomial_lcm(LMF, LMG) # This fails
LCM12 = lcm(LMf, LMg)
LCM12


 3  
x ⋅y

and the $s$-polynomial can hence be calculated thusly:

In [145]:
def s_polynomial(f, g):
    LCM_fg = lcm(LM(f), LM(g))
    s = LCM_fg*(f/LT(f)-g/LT(g))
    return s.expand()
s=s_polynomial(F[0], F[1])
s

  2
-x 

Now we need to divide this $s$ value by all the terms in $F$, write it out in terms of quotients and divisors and then consider the remainder, so for example:

In [146]:
display(div(s, F[0]))
display(div(s, F[1]))

⎛     2⎞
⎝0, -x ⎠

⎛     2⎞
⎝0, -x ⎠

\begin{align*}
 & -x^{2} & = & 0\left(2xy\right) & -x^{2}\\
+ &  & =\\
 & -x^{2} & = & 0\left(2y^{2}-x^{2}y\right) & -x^{2}\\
\hline  & -2x^{2} & = & 0\left(2xy\right)+0\left(2y^{2}-x^{2}y\right) & -2x^{2}\\
\implies & r & = & \mathrm{mean}\left(r\in R\right)
\end{align*}


And so this remainder value can be calculated, in `sympy`, like so:

In [148]:
r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
r = sum(r_list)/len(r_list)
r

  2
-x 

Now because this remainder value is non-zero, it belongs to the Groebner Bases and it needs to be added back to $F$ and the process needs to start again.

### Continue testing values

In [ ]:
The value

In [ ]:
Note that we have to average the values, because:

In [ ]:
Hmmm, this is't right, should this maybe be

In [5]:
display(g(x))

    2    2       2  2       2           2                        
19⋅t ⋅y⋅z  + 17⋅y ⋅z  + 11⋅y ⋅z + 13⋅y⋅z  + 7⋅y⋅z + 3⋅y + 5⋅z + 2

In [430]:
srepr(g(x))

"Add(Mul(Integer(17), Pow(Symbol('x'), Integer(2)), Pow(Symbol('y'), Integer(2))), Mul(Integer(11), Pow(Symbol('x'), Integer(2)), Symbol('y')), Mul(Integer(19), Symbol('x'), Pow(Symbol('y'), Integer(2)), Pow(Symbol('z'), Integer(2))), Mul(Integer(13), Symbol('x'), Pow(Symbol('y'), Integer(2))), Mul(Integer(7), Symbol('x'), Symbol('y')), Mul(Integer(3), Symbol('x')), Mul(Integer(5), Symbol('y')), Integer(2))"

In [431]:
pprint(g(x))

    2  2       2           2  2         2                        
17⋅x ⋅y  + 11⋅x ⋅y + 19⋅x⋅y ⋅z  + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2


In [392]:
f(x)

    2  2  2       2  2       2           2                        
19⋅x ⋅y ⋅z  + 17⋅x ⋅y  + 11⋅x ⋅y + 13⋅x⋅y  + 7⋅x⋅y + 3⋅x + 5⋅y + 2

Notice how the order is lexical $\uparrow$

The **multidegree** is the largest power ocross all variables

In [393]:
def multideg(polynomial):
    return max(degree_list(polynomial))
multideg(f(x))

2

Occassionally this will be written as $\left\langle 2,2,2\right\rangle$

The *Leading Coefficient* is the coefficient corresponding to the Leading Monomial

In [394]:
LC(f(x))
# Can also be used as a method
# poly(f(x), domain = 'QQ').LC()

19

The leading monomial is the corresponding monomial

In [395]:
# Also known as the initial.
LM(f(x))

 2  2  2
x ⋅y ⋅z 

The leading term is the largest term, given the order

In [396]:
LT(f(x))

    2  2  2
19⋅x ⋅y ⋅z 

## S Polynomial

The $S$-Polynomial is a polynomial used in Buchberger's Algorithm, given some polynomials:

In [397]:
def f(x, y):
    return x**3*y**2-x**2*y**3+x
def g(x, y):
    return 3*x**4*y+y**2

f, g = f(x, y), g(x, y)    

The Lowest Common Multiple of the leading monomials can be given by:

In [399]:
from sympy.polys.monomials import monomial_mul, monomial_lcm, monomial_divides, term_div
# https://docs.sympy.org/latest/modules/polys/basics.html
LMf = LM(f)
LMg = LM(g)
# LCM12 = monomial_lcm(LMF, LMG) # This fails
LCM12 = lcm(LMf, LMg)
LCM12


 4  2
x ⋅y 

Then the $S$-polymial is given by:

In [400]:
def s_polynomial(f, g):
    LCM_fg = lcm(LM(f), LM(g))
    s = LCM_fg*(f/LT(f)-g/LT(g))
    return s.expand()
s=s_polynomial(f, g)
s

                3
   3  3    2   y 
- x ⋅y  + x  - ──
               3 

This can be used to determine if a polynomial belongs to a Groebner Bases ($G$) or not, this is known as Buchberger's Criterion. If the remainder of the S polynomial divided by all elements of $F$ is 0, the pair belongs in $G$, if not, the remainder belongs in $G$.

consider the set of functions:

In [401]:
F = [2*x-y, x**2-1-y]
F

⎡          2        ⎤
⎣2⋅x - y, x  - y - 1⎦

To determine if this is a Groebner Basis:

In [402]:
s=s_polynomial(F[0], F[1])
s

  x⋅y        
- ─── + y + 1
   2         

In [403]:
q,r = div(s,f*g)
r

  x⋅y        
- ─── + y + 1
   2         

This is non-zero and so this should be added back to $F$, however we should be more careful to calculate the remainder appropriately:

In [404]:
print(div(s, F[0]))
print(div(s, F[1]))

(-y/4, -y**2/4 + y + 1)
(0, -x*y/2 + y + 1)


This implies that:

$$
-\frac{xy}{2}+y+1 = -\frac{y}{4} (2x-y) + 0 (x^2-y-1) + (-\frac{y^2}{4} +y + 1 - \frac{xy}{2} + y + 1)
$$

So to calculate the remainder more robustly:

In [405]:
r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
r = sum(r_list)
r

         2          
  x⋅y   y           
- ─── - ── + 2⋅y + 2
   2    4           

And adding that back in:

In [406]:
F.append(r)

Now the Remainder can be calculated again for $f_1$ and $f_3$

In [407]:
s=s_polynomial(F[0], F[2])
q,r = div(s,f*g)
r

   2          
- y  + 4⋅y + 4

As this is non-zero it should be added back in

In [408]:
F.append(r)
F

⎡                              2                          ⎤
⎢          2            x⋅y   y                2          ⎥
⎢2⋅x - y, x  - y - 1, - ─── - ── + 2⋅y + 2, - y  + 4⋅y + 4⎥
⎣                        2    4                           ⎦

Now comparing $f_1$ and $f_4$

In [409]:
s=s_polynomial(F[0], F[3])
q,r = div(s,f*g)
if r!=0:
    F.append(r)
    print(F)
else:
    print("This pair is a Groebner Basis")

[2*x - y, x**2 - y - 1, -x*y/2 - y**2/4 + 2*y + 2, -y**2 + 4*y + 4, 4*x*y + 4*x - y**3/2]


Now comparing $f_1$ and $f_5$

In [410]:
s=s_polynomial(F[0], F[4])
q,r = div(s,f*g)
r
if r!=0:
    F.append(r)
    print(F)
else:
    print("This pair is a Groebner Basis")

[2*x - y, x**2 - y - 1, -x*y/2 - y**2/4 + 2*y + 2, -y**2 + 4*y + 4, 4*x*y + 4*x - y**3/2, -x + y**3/8 - y**2/2]


Hmmm, this doesn't seem to be terminating, I'm going to have to put more research into this.

In [411]:
f = x**3-2*x*y
g = x**2*y+x-2*y**2
F = [f, g]

In [412]:
def poly_prod(F):
    product_val = 1
    for poly in F:
        product_val *= poly
    return product_val

In [413]:
def remainder_set_div(s, F):
    r_list = [ div(s, F[i])[1] for i in range(len(F)) ]
    r = sum(r_list)
    return r


In [414]:
def buchberger_criterion(f,g, F):
    s = s_polynomial(f, g)
    r = remainder_set_div(s, F)
    return r

In [415]:
r = buchberger_criterion(F[0], F[1], F)
if r != 0:
    F.append(r)


In [416]:
F

⎡ 3           2            2      2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x ⎦

So we got back a remainder of $-x^2$, this isn't zero, so we throw it in the bag

We haven't tried every combination, so onto the next one:

In [417]:
r = buchberger_criterion(F[0], F[2], F)
if r != 0:
    F.append(r)


In [418]:
F

⎡ 3           2            2      2        ⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x , -6⋅x⋅y⎦

This gave a remainder of $-2xy$, so it goes in the bag

Now we try the next combination $f_1$ and $f_4$

In [419]:
r = buchberger_criterion(F[0], F[3], F)
if r != 0:
    F.append(r)
F

⎡ 3           2            2      2                2⎤
⎣x  - 2⋅x⋅y, x ⋅y + x - 2⋅y , -2⋅x , -6⋅x⋅y, -6⋅x⋅y ⎦

So the problem at this step is that diving by the product of F is not what we want

In [421]:
print(div(-2*x*y**2, F[1]))
print(div(-2*x*y**2, F[2]))
print(div(-2*x*y**2, F[3]))
print(div(-2*x*y**2, F[4]))

(0, -2*x*y**2)
(0, -2*x*y**2)
(y/3, 0)
(1/3, 0)
